In [1]:
import ast
import json
import warnings
import os
import pandas as pd
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.metrics import accuracy_score
from snowflake.ml.modeling.model_selection import GridSearchCV
from snowflake.ml.modeling.preprocessing import OneHotEncoder
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.registry import Registry
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session
from snowflake.snowpark import types as T
from snowflake.snowpark.functions import col

warnings.simplefilter(action="ignore", category=UserWarning)

In [2]:
connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "role": os.getenv("SNOWFLAKE_ROLE"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
}

session = Session.builder.configs(connection_parameters).create()


In [3]:

print(f"Current Database and schema: {session.get_fully_qualified_current_schema()}")
print(f"Current Warehouse: {session.get_current_warehouse()}")

Current Database and schema: "MLOPS"."ADVERTISING"
Current Warehouse: "COMPUTE_WH"


In [4]:
ad_df = session.table("ADVERTISING")


In [5]:
ad_df.show()

-------------------------------------------
|"TV"   |"RADIO"  |"NEWSPAPER"  |"SALES"  |
-------------------------------------------
|230.1  |37.8     |69.2         |22.1     |
|44.5   |39.3     |45.1         |10.4     |
|17.2   |45.9     |69.3         |12.0     |
|151.5  |41.3     |58.5         |16.5     |
|180.8  |10.8     |58.4         |17.9     |
|8.7    |48.9     |75.0         |7.2      |
|57.5   |32.8     |23.5         |11.8     |
|120.2  |19.6     |11.6         |13.2     |
|8.6    |2.1      |1.0          |4.8      |
|199.8  |2.6      |21.2         |15.6     |
-------------------------------------------



In [6]:
train_df, test_df = ad_df.random_split(weights=[0.8, 0.2], seed=8)

In [7]:
session.sql(
    f"ALTER WAREHOUSE {session.get_current_warehouse()[1:-1]} SET WAREHOUSE_SIZE=LARGE;"
).collect()

[Row(status='Statement executed successfully.')]

In [8]:
parameter_grid = [
    {'n_estimators': 250, 'max_depth': 45, 'learning_rate': 0.01, 'subsample': 0.8, 'colsample_bytree': 0.8},
    {'n_estimators': 300, 'max_depth': 50, 'learning_rate': 0.01, 'subsample': 0.8, 'colsample_bytree': 0.9},
    {'n_estimators': 350, 'max_depth': 70, 'learning_rate': 0.01, 'subsample': 0.8, 'colsample_bytree': 0.9},
    {'n_estimators': 400, 'max_depth': 100, 'learning_rate': 0.01, 'subsample': 1, 'colsample_bytree': 0.9},
    {'n_estimators': 600, 'max_depth': 150, 'learning_rate': 0.01, 'subsample': 1, 'colsample_bytree': 0.9},
]


In [20]:
from snowflake.ml.modeling.xgboost import XGBRegressor

def train_and_evaluate(params, train_df, test_df):
    model = XGBRegressor(
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        input_cols=['TV', 'RADIO', 'NEWSPAPER'],
        label_cols='SALES', 
        output_cols='predicted_sales'
    )
    model.fit(train_df)
    r2_score = model.score(test_df)
    return model, r2_score

results = []
for params in parameter_grid:
    model, r2 = train_and_evaluate(params, train_df, test_df)
    results.append((params, r2, model))

for result in results:
    params, score, _ = result
    param_str = ', '.join(f"{k}={v}" for k, v in params.items())
    print(f"Parameters: {param_str} -> R2 Score: {score}")


Parameters: n_estimators=250, max_depth=45, learning_rate=0.01, subsample=0.8, colsample_bytree=0.8 -> R2 Score: 0.766387804819972
Parameters: n_estimators=300, max_depth=50, learning_rate=0.01, subsample=0.8, colsample_bytree=0.9 -> R2 Score: 0.8402931392007842
Parameters: n_estimators=350, max_depth=70, learning_rate=0.01, subsample=0.8, colsample_bytree=0.9 -> R2 Score: 0.8772703692786791
Parameters: n_estimators=400, max_depth=100, learning_rate=0.01, subsample=1, colsample_bytree=0.9 -> R2 Score: 0.8951463194303564
Parameters: n_estimators=600, max_depth=150, learning_rate=0.01, subsample=1, colsample_bytree=0.9 -> R2 Score: 0.916201064193848


In [21]:
best_params, best_score, best_model = max(results, key=lambda x: x[1])
best_param_str = ', '.join(f"{k}={v}" for k, v in best_params.items())
print(f"Best Parameters: {best_param_str}")
print(f"Best R2 Score: {best_score}")

Best Parameters: n_estimators=600, max_depth=150, learning_rate=0.01, subsample=1, colsample_bytree=0.9
Best R2 Score: 0.916201064193848


In [22]:
predictions = best_model.predict(test_df)

In [23]:
predictions.show()

----------------------------------------------------------------
|"TV"   |"RADIO"  |"NEWSPAPER"  |"SALES"  |"predicted_sales"   |
----------------------------------------------------------------
|8.7    |48.9     |75.0         |7.2      |11.234919548034668  |
|57.5   |32.8     |23.5         |11.8     |11.47921371459961   |
|120.2  |19.6     |11.6         |13.2     |13.154800415039062  |
|199.8  |2.6      |21.2         |15.6     |16.5615177154541    |
|97.5   |7.6      |7.2          |13.7     |12.398638725280762  |
|195.4  |47.7     |52.9         |22.4     |22.05717658996582   |
|62.3   |12.6     |18.3         |9.7      |9.181100845336914   |
|70.6   |16.0     |40.8         |10.5     |11.217970848083496  |
|112.9  |17.4     |38.6         |11.9     |12.820030212402344  |
|95.7   |1.4      |7.4          |11.9     |12.086431503295898  |
----------------------------------------------------------------



In [ ]:
session.sql(
    f"ALTER WAREHOUSE {session.get_current_warehouse()[1:-1]} SET WAREHOUSE_SIZE=XSMALL;"
).collect()

[Row(status='Statement executed successfully.')]

###  Log the Model to the Registry

In [24]:
reg = Registry(session=session)



In [25]:
model_version = reg.log_model(
    model=best_model,
    model_name="XGBRegressorModel",
    version_name="v1",
    conda_dependencies=["xgboost", "pandas", "numpy"],  
    comment="Initial version of the XGB Regressor model",
    metrics={"R2 Score": best_score}  # Assuming `best_score` is available from your training
)


## Use the Model for Predictions

In [30]:
model_ref = reg.get_model("XGBRegressorModel")

